In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
def load_utt_ntt():
    ntt_path = "../../corpus/NTT/"
    utt_list = []
    for file_ in os.listdir(ntt_path):
        if not "json" in file_:
            continue 
        with open(ntt_path+file_, "r",  encoding="utf-8") as f:
            convs = json.load(f)
            for did in convs["convs"]:
                dids = list( did.keys() )[0]
                conv = did[dids]
                # conv = did[dids][3::3]
                utt_list.extend( [ utt for utt in conv])
    
    print(len(utt_list))
    return utt_list

In [3]:
c_path = "../X_y_data/corpus/"
c_name = "ntt_sentence_docs.pickle"
c_name = "ntt_sentence_docs_plain.pickle"
corpusM = DataManager(c_path)

if os.path.exists(c_path+c_name):
    docs = corpusM.load_data(c_name)
    ntt_utt = list(map(str, docs))
else:
    ntt_utt_ = load_utt_ntt()
    docs = sentence2docs(ntt_utt_[:1000])
    ntt_utt = list(map(str, docs))
    corpusM.save_data(c_name, docs)

success load : ../X_y_data/corpus/ntt_sentence_docs_plain.pickle


In [4]:
t = "この時期から資格を取りますねぇ"

In [5]:
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [6]:
import copy
def predicate_argument_structure_analysis(text, clean_=True):
    if clean_:
        text = clean_text(text)
    doc = nlp(text)
    triple_list = []
    suit_list = []
    for i, token in enumerate( doc ):
        if token.pos_ in ["VERB", "ADJ"]:
            # 受動表現の可能性があるものは回避
            if i<=len(doc)-2 and doc[i+1].pos_ == "AUX" and doc[i+1].lemma_ in ["れる", "られる"]:
                # print(doc)
                continue
            is_exist_case = False
            for c in token.children:
                if c.dep_ in ["nsubj", "obj", "obl"]:
                    noun = c.lemma_
                    for c2 in c.children:
                        if c2.dep_ == "case":
                            is_exist_case = True
                        # if c2.dep_ == "case":
                            case = c2.orth_
                            if case == "は":
                                case = "が"
                            triple_list.append( (noun, case, token.lemma_ ))
             
    return triple_list
    # return suit_list

In [12]:
def dependency_suit(text):
    # 解析が必要か否か
    if isinstance(text, spacy.tokens.doc.Doc):
        doc = text
    elif isinstance(text, str):
        doc = nlp(text)
    suit_list = []
    for i, token in enumerate( doc ):
        if token.dep_!="punct" and token.head:
            suit_list.append([token.orth_, token.dep_, token.head.lemma_])
    return suit_list


In [18]:
def recursive_children(token:spacy.tokens.token.Token):
    pass

In [19]:
def dependency_tree(text):
    if isinstance(text, spacy.tokens.doc.Doc):
        doc = text
    elif isinstance(text, str):
        doc = nlp(text)
    root = None
    # ルートの取得

    for i, token in enumerate( doc ):
        if token.dep_=="ROOT" :
            root = token
            print(type(root))
    for i, c in enumerate(root.children):
        print(c)

In [20]:
# for doc in docs:
#     # print(predicate_argument_structure_analysis(utt, clean_=False))
#     print(dependency_suit(doc))

In [21]:
dependency_tree(t+"。")

<class 'spacy.tokens.token.Token'>
時期
資格
ます
ねぇ
。


In [11]:
from gensim.models import KeyedVectors

w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[277] 2022-02-06 09:26:51,562 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[277] 2022-02-06 09:27:53,186 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-02-06T09:27:53.185870', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}
